<br>

<img src="https://i.ibb.co/tYfwcwC/header-1.png" width="100%">

<br>

---

<br>

<center><h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;">EXPLORATORY DATA ANALSYSIS AND SOLUTIONING</h2><h4 style="font-family: Verdana; font-size: 18px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;">TOWARDS BRAIN TUMOR RADIOGENOMIC CLASSIFICATION</h4></center>

<br>
    
<h5 style="text-align: center; font-family: Verdana; font-size: 14px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5><br>

---

<br>

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <br><center><b style="font-size: 16px;">🛑🛑🛑 &nbsp; CAUTION:  THIS NOTEBOOK IS A WORK IN PROGRESS  &nbsp; 🛑🛑🛑</b></center><br>
</div>

<br>


---
<br>

<br>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;">TABLE OF CONTENTS</h2>

---

<h2 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h2>

---

<h2 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h2>

---

<h2 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h2>

---

<h2 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h2>

---


<br>

<a id="imports"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: #CD0001;" id="imports">0&nbsp;&nbsp;IMPORTS</h1>

In [ ]:
print("\n... PIP/APT INSTALLS STARTING ...")
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
!rm -rf ./gdcm*
!pip install efficientnet-3D
!pip install git+https://github.com/qubvel/classification_models.git
!pip install -U --pre efficientnet

# !conda install -c conda-forge gdcm -y
!pip install pandarallel
print("... PIP/APT INSTALLS COMPLETE ...\n")


print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None; pd.set_option('max_columns', 100);
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import efficientnet_3D.tfkeras as efn

# Other Competition Related Imports
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pandarallel import pandarallel; pandarallel.initialize();

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import scipy
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib import animation, rc; rc('animation', html='jshtml')
import matplotlib.patches as patches

import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2
print("\n\n... IMPORTS COMPLETE ...\n")
    

print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...")
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()
print("... SEEDING COMPLETE ...\n\n")

print("\n... SETTING PRESETS STARTING...")
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
label_map_int_2_str = {0: "MGMT_NEGATIVE", 1: "MGMT_POSITIVE"}
label_map_str_2_int = {"MGMT_NEGATIVE":0, "MGMT_POSITIVE":1}
print("... SETTING PRESETS COMPLETE...\n\n")

<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION</h1>

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">1.1  OVERVIEW</h2>

---

<b style="text-decoration: underline; font-family: Verdana;">COMPETITION DESCRIPTION</b>

A malignant tumor in the brain is a life-threatening condition. Known as glioblastoma, it's both the most common form of brain cancer in adults and the one with the worst prognosis, with median survival being less than a year. The presence of a specific genetic sequence in the tumor known as MGMT promoter methylation has been shown to be a favorable prognostic factor and a strong predictor of responsiveness to chemotherapy.

Currently, genetic analysis of cancer requires surgery to extract a tissue sample. Then it can take several weeks to determine the genetic characterization of the tumor. Depending upon the results and type of initial therapy chosen, a subsequent surgery may be necessary. If an accurate method to predict the genetics of the cancer through imaging (i.e., radiogenomics) alone could be developed, this would potentially minimize the number of surgeries and refine the type of therapy required.

The Radiological Society of North America (RSNA) has teamed up with the Medical Image Computing and Computer Assisted Intervention Society (the MICCAI Society) to improve diagnosis and treatment planning for patients with glioblastoma. In this competition you will predict the genetic subtype of glioblastoma using MRI (magnetic resonance imaging) scans to train and test your model to detect for the presence of MGMT promoter methylation.

If successful, you'll help brain cancer patients receive less invasive diagnoses and treatments. The introduction of new and customized treatment strategies before surgery has the potential to improve the management, survival, and prospects of patients with brain cancer.

**Secondary Description From UPenn**
> The participants are called to use the provided mpMRI data to extract imaging/radiomic features that they consider appropriate, and analyze them through machine learning algorithms, in an attempt to predict the MGMT promoter methylation status. The participants do not need to be limited to volumetric parameters, but can also consider intensity, morphologic, histogram-based, and textural features, as well as spatial information, deep learning features, and glioma diffusion properties extracted from glioma growth models.

> Note that participants will be evaluated for the predicted MGMT status of the subjects indicated in the accompanying spreadsheet.

<br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <br><center><b style="font-size: 16px;">🤔🤔🤔 &nbsp; MY INTERPRETATION OF THIS COMPETITION &nbsp; 🤔🤔🤔</b></center><br><br><b style="font-size: 12px; color: black;">In this competition we are tasked with identifying/predicting the <i>genetic subtype <font color="darkred">(genetic subtype = a group of tumors that is enriched for genetic aberrations in a set of subtype predictor genes)</font></i> of <i>glioblastoma <font color="darkred">(glioblastoma = brain tumor)</font></i><br><br>HUH?!?!?<br><br>All this means is that we are taking in dicom images containing 3D representations (slices) of a patients brain which we will process with computer vision algorithms to allow us to perform binary classification. The binary classification is to identify if, within the patients image data, MGMT promoter methylation is present. The diagram below shows a simple, if slightly inaccurate, representation of what is required.</b><br><br><center><img src="https://www.researchgate.net/profile/Qi-Dou-2/publication/313019762/figure/fig2/AS:691329537433601@1541837214995/The-hierarchical-architecture-of-the-3D-CNN-model.png" width=50%></center><br>
</div>

---

<br>

<b style="text-decoration: underline; font-family: Verdana;">SUBMISSION EVALUATION/RESTRICTIONS AND FILE FORMAT</b>

**Submission Evaluation**
* Submissions are evaluated on the [area under the ROC curve](http://en.wikipedia.org/wiki/Receiver_operating_characteristic) between the predicted probability and the observed target.

<br>

**Submission Restrictions**
* **THIS IS A KERNELS ONLY COMPETITION**
    * Submissions to this competition must be made through Notebooks. 
    * In order for the "Submit" button to be active after a commit, the following conditions must be met:
        * *CPU Notebook <= 9 hours run-time*
        * *GPU Notebook <= 9 hours run-time*
        * *Internet access disabled*
        * *Freely & publicly available external data is allowed, including pre-trained models*
        * *Submission file must be named `submission.csv`*

<br>

**Submission File Format**
* For each **`BraTS21ID`** in the test set, you must predict a probability for the target **`MGMT_value`**. The file should contain a header and have the following format:
>```
>BraTS21ID,MGMT_value
>00001,0.5
>00013,0.999
>00015,0.1
>etc.
>```

<br>

---

<br>

<b style="text-decoration: underline; font-family: Verdana;">COMPETITION TIMELINE</b>

* **July 13, 2021** - Start Date.
* **October 8, 2021** - Entry Deadline. You must accept the competition rules before this date in order to compete.
* **October 8, 2021** - Team Merger Deadline. This is the last day participants may join or merge teams.
* **October 15, 2021** - Final Submission Deadline.
* **October 25, 2021** - Winners’ Requirements Deadline. This is the deadline for winners to submit to the host/Kaggle their training code, video, method description.

> All deadlines are at 11:59 PM UTC on the corresponding day unless otherwise noted. The competition organizers reserve the right to update the contest timeline if they deem it necessary.



<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">1.2  DATA DESCRIPTION</h2>

---

<br>

<b style="text-decoration: underline; font-family: Verdana;">DATA SPLITS/COHORTS</b>

The competition data is defined by three cohorts: **Training**, **Validation (Public)**, and **Testing (Private)**. 
* The **“Training”** and the **“Validation”** cohorts are provided to the participants
* The **“Testing”** cohort is kept hidden at all times, during and after the competition

These 3 cohorts are structured as follows: 
* Each independent case has a **dedicated folder identified by a five-digit number**.
* Within each of these **“case”** folders, there are four sub-folders
    * Each of these **"case"** subfolders corresponds to each of the structural **m**ulti-**p**arametric **MRI** (**mpMRI**) scans, in **DICOM** format. 
    * The exact mpMRI scans included are:
        * Fluid Attenuated Inversion Recovery (FLAIR)
        * T1-weighted pre-contrast (T1w)
        * T1-weighted post-contrast (T1Gd)
        * T2-weighted (T2)

Exact folder structure:

```
Training/Validation/Testing
│
└─── 00000
│   │
│   └─── FLAIR
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T1w
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T1wCE
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T2w
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ .....
│   
└─── 00001
│   │ ...
│   
│ ...   
│   
└─── 00002
│   │ ...

```

<b style="text-decoration: underline; font-family: Verdana;">FILES</b>

**`train/`** 
- folder containing the training files, with each top-level folder representing a subject

**`train_labels.csv`** 
- file containing the target MGMT_value for each subject in the training data (e.g. the presence of MGMT promoter methylation)

**`test/`** 
- the test files, which use the same structure as train/; your task is to predict the MGMT_value for each subject in the test data. NOTE: the total size of the rerun test set (Public and Private) is ~5x the size of the Public test set

**`sample_submission.csv`** 
- a sample submission file in the correct format

<br>

<a id="setup"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="setup">2&nbsp;&nbsp;SETUP</h1>

In [ ]:
ROOT_DIR = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification"

TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TEST_DIR = os.path.join(ROOT_DIR, "test")

SS_CSV = os.path.join(ROOT_DIR, "sample_submission.csv")
TRAIN_CSV = os.path.join(ROOT_DIR, "train_labels.csv")

train_df = pd.read_csv(TRAIN_CSV)
train_df["path_to_flair_dir"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_DIR, f"{x:>05}", "FLAIR"))
train_df["flair_image_count"] = train_df.path_to_flair_dir.progress_apply(lambda x: len(os.listdir(x)))
train_df["path_to_t1w_dir"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_DIR, f"{x:>05}", "T1w"))
train_df["t1w_image_count"] = train_df.path_to_t1w_dir.progress_apply(lambda x: len(os.listdir(x)))
train_df["path_to_t1wce_dir"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_DIR, f"{x:>05}", "T1wCE"))
train_df["t1wce_image_count"] = train_df.path_to_t1wce_dir.progress_apply(lambda x: len(os.listdir(x)))
train_df["path_to_t2w_dir"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_DIR, f"{x:>05}", "T2w"))
train_df["t2w_image_count"] = train_df.path_to_t2w_dir.progress_apply(lambda x: len(os.listdir(x)))
                                                                  
ss_df = pd.read_csv(SS_CSV)
ss_df["path_to_flair_dir"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_DIR, f"{x:>05}", "FLAIR"))
ss_df["flair_image_count"] = ss_df.path_to_flair_dir.progress_apply(lambda x: len(os.listdir(x)))
ss_df["path_to_t1w_dir"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_DIR, f"{x:>05}", "T1w"))
ss_df["t1w_image_count"] = ss_df.path_to_t1w_dir.progress_apply(lambda x: len(os.listdir(x)))
ss_df["path_to_t1wce_dir"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_DIR, f"{x:>05}", "T1wCE"))
ss_df["t1wce_image_count"] = ss_df.path_to_t1wce_dir.progress_apply(lambda x: len(os.listdir(x)))
ss_df["path_to_t2w_dir"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_DIR, f"{x:>05}", "T2w"))
ss_df["t2w_image_count"] = ss_df.path_to_t2w_dir.progress_apply(lambda x: len(os.listdir(x)))

print("\n\nTRAIN DATAFRAME\n")
display(train_df.head())

print("\n\n\nSAMPLE SUBMISSION DATAFRAME\n")
display(ss_df.head())

print("\nCOUNTS PER SCAN TYPE\n")
for c_name in [c for c in train_df.columns if "count" in c]:
    print(c_name, print(train_df[c_name].sum()))

<br>

<a id="helper_functions"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="helper_functions">3&nbsp;&nbsp;HELPER FUNCTIONS</h1>

In [ ]:
def get_list_of_dcm_paths(dir_path):
    return sorted([os.path.join(dir_path, f_name) for f_name in os.listdir(dir_path)], key=lambda x: int(x.rsplit("-", 1)[1].split(".", 1)[0]))

def dicom2array(path, voi_lut=True, fix_monochrome=True):
    """ Convert dicom file to numpy array 
    
    Args:
        path (str): Path to the dicom file to be converted
        voi_lut (bool): Whether or not VOI LUT is available
        fix_monochrome (bool): Whether or not to apply monochrome fix
        
    Returns:
        Numpy array of the respective dicom file 
        
    """
    # Use the pydicom library to read the dicom file
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to 
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # The XRAY may look inverted
    #   - If we want to fix this we can
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    # Normalize the image array and return
    data = (data-np.min(data))/(np.max(data)-np.min(data))
    return data

def create_animation(np_arr):
    fig = plt.figure(figsize=(5, 5))
    plt.axis('off')
    im = plt.imshow(np_arr[..., 0], cmap="bone")
    plt.title("mpMRI Animation", fontweight="bold")
    def animate_func(i):
        im.set_array(np_arr[..., i])
        return [im]
    plt.close()
    return animation.FuncAnimation(fig, animate_func, frames = np_arr.shape[-1], interval = 1000//24)


def create_4scan_animation(flair_arr, t1w_arr, t1wce_arr, t2w_arr):
    
    def animate_func(i):
        flair_im.set_array(flair_arr[..., i])
        t1w_im.set_array(t1w_arr[..., i])
        t1wce_im.set_array(t1wce_arr[..., i])
        t2w_im.set_array(t2w_arr[..., i])
        return [flair_im, t1w_im, t1wce_im, t2w_im]
    
    fig = plt.figure(figsize=(18, 5))
    
    #
    plt.subplot(1,4,1)
    plt.axis('off')
    flair_im = plt.imshow(flair_arr[..., 0], cmap="bone")
    plt.title("FLAIR Animation", fontweight="bold")
    
    #
    plt.subplot(1,4,2)
    plt.axis('off')
    t1w_im = plt.imshow(t1w_arr[..., 0], cmap="bone")
    plt.title("T1w Animation", fontweight="bold")
    
    #
    plt.subplot(1,4,3)
    plt.axis('off')
    t1wce_im = plt.imshow(t1wce_arr[..., 0], cmap="bone")
    plt.title("T1wCE Animation", fontweight="bold")

    
    #
    plt.subplot(1,4,4)
    plt.axis('off')
    t2w_im = plt.imshow(t2w_arr[..., 0], cmap="bone")
    plt.title("T2w Animation", fontweight="bold")
    
    plt.close()
    
    return animation.FuncAnimation(fig, animate_func, frames = flair_arr.shape[-1], interval = 1000//16)


def get_dicom_meta(row, attrs):
    dcm_file = pydicom.read_file(row.dcm_path)
    for val in attrs:
        row[val] = dcm_file.get(val, None)
    return row


def load_npz(np_file_path, is_tf=False):
    if is_tf:
        return np.load(np_file_path.numpy().decode())["arr_0"] 
    else:
        return np.load(np_file_path)["arr_0"] 
    
    
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

<br>

<a id="helper_functions"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="dicom_understanding">4&nbsp;&nbsp;DICOM UNDERSTANDING</h1>

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.0 SET EXAMPLE PATIENT DATA</h2>

---

<br>


In [ ]:
EX_NUMBER = 0
EX_SLICE = 0
EX_PATIENT = train_df.iloc[EX_NUMBER]

ALL_DICOM_PATHS = glob(os.path.join(TRAIN_DIR, "**/*.dcm"), recursive=True)
FLAIR_DCM_PATHS = get_list_of_dcm_paths(EX_PATIENT["path_to_flair_dir"])
T1W_DCM_PATHS = get_list_of_dcm_paths(EX_PATIENT["path_to_t1w_dir"])
T1WCE_DCM_PATHS = get_list_of_dcm_paths(EX_PATIENT["path_to_t1wce_dir"])
T2W_DCM_PATHS = get_list_of_dcm_paths(EX_PATIENT["path_to_t2w_dir"])

FLAIR_EXAMPLE_DCM_1 = FLAIR_DCM_PATHS[EX_SLICE]
FLAIR_EXAMPLE_DCM_2 = FLAIR_DCM_PATHS[EX_SLICE+1]
T1W_EXAMPLE_DCM_1 = T1W_DCM_PATHS[EX_SLICE]
T1W_EXAMPLE_DCM_2 = T1W_DCM_PATHS[EX_SLICE+1]
T1WCE_EXAMPLE_DCM_1 = T1WCE_DCM_PATHS[EX_SLICE]
T1WCE_EXAMPLE_DCM_2 = T1WCE_DCM_PATHS[EX_SLICE+1]
T2W_EXAMPLE_DCM_1 = T2W_DCM_PATHS[EX_SLICE]
T2W_EXAMPLE_DCM_2 = T2W_DCM_PATHS[EX_SLICE+1]

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.1  BASIC STRUCTURE OF FLAIR DICOM</h2>

---

<br>


In [ ]:
flair_ex_dcm_1 = pydicom.read_file(FLAIR_EXAMPLE_DCM_1)
flair_ex_dcm_2 = pydicom.read_file(FLAIR_EXAMPLE_DCM_2)
FLAIR_ATTRS = flair_ex_dcm_1.dir()

print("FLAIR ATTRIBUTES:")
for k in FLAIR_ATTRS: print(f"\t--> {k:<30}( Same Across Dicoms = {str(flair_ex_dcm_1.get(k)==flair_ex_dcm_2.get(k)):<5} )")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.2  BASIC STRUCTURE OF T1W DICOM</h2>

---

<br>


In [ ]:
t1w_ex_dcm_1 = pydicom.read_file(T1W_EXAMPLE_DCM_1)
t1w_ex_dcm_2 = pydicom.read_file(T1W_EXAMPLE_DCM_2)
T1W_ATTRS = t1w_ex_dcm_1.dir()

print("T1W ATTRIBUTES:")
for k in T1W_ATTRS: print(f"\t--> {k:<30}( Same Across Two Dicoms = {str(t1w_ex_dcm_1.get(k)==t1w_ex_dcm_2.get(k)):<5} )")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.3  BASIC STRUCTURE OF T1WCE DICOM</h2>

---

<br>

This dicom has an additional two fields that the other dicoms don't have:
* **`InPlanePhaseEncodingDirection`**
* **`InStackPositionNumber`**

In [ ]:
t1wce_ex_dcm_1 = pydicom.read_file(T1WCE_EXAMPLE_DCM_1)
t1wce_ex_dcm_2 = pydicom.read_file(T1WCE_EXAMPLE_DCM_2)
T1WCE_ATTRS = t1wce_ex_dcm_1.dir()

print("T1WCE ATTRIBUTES:")
for k in T1WCE_ATTRS: print(f"\t--> {k:<30}( Same Across Two Dicoms = {str(t1wce_ex_dcm_1.get(k)==t1wce_ex_dcm_2.get(k)):<5} )")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.4  BASIC STRUCTURE OF T2W DICOM</h2>

---

<br>


In [ ]:
t2w_ex_dcm_1 = pydicom.read_file(T2W_EXAMPLE_DCM_1)
t2w_ex_dcm_2 = pydicom.read_file(T2W_EXAMPLE_DCM_2)
T2W_ATTRS = t2w_ex_dcm_1.dir()

print("T2W ATTRIBUTES:")
for k in T2W_ATTRS: print(f"\t--> {k:<30}( Same Across Two Dicoms = {str(t2w_ex_dcm_1.get(k)==t2w_ex_dcm_2.get(k)):<5} )")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.5  CAPTURE ALL OPTIONS AND VALUE COUNTS FOR DICOM</h2>

---

<br>

We will not be capturing dicom metadata for UIDs or PixelData (this would cause OOM issues).

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <br><center><b style="font-size: 16px;">⏱⏱⏱ &nbsp; NOTE: THIS CELL WILL TAKE ~15 MINUTES TO RUN  &nbsp; ⏱⏱⏱</b></center><br>
</div>


In [ ]:
t1 = time.time()

DESIRED_ATTRS = [x for x in T1WCE_ATTRS if (x!="PixelData" and "UID" not in x)]
dicom_df = pd.DataFrame({"dcm_path":ALL_DICOM_PATHS})
dicom_df = dicom_df.reindex(columns=(["dcm_path",]+DESIRED_ATTRS))

print(f"\n ... Processing Started...")

# Regular method
#    - for 10,000 dicom files the regular takes 94.639 seconds
#    - processing will take (350000/10000)*94.639 ~= 3325 seconds ~= 56 minutes (actual is 71 minutes)

##### dicom_df = dicom_df.progress_apply(lambda x: get_dicom_meta(x, DESIRED_ATTRS), axis=1)

# Use Multiprocessing (Pandarallel)
#    - no progress bar but approximately 3.75 times faster
#    - for 10,000 dicom files pandarallel takes 25.529 seconds
#    - processing will take (350000/10000)*25.529 ~= 893 seconds ~= 15 minutes (actual is 19 minutes)
dicom_df = dicom_df.parallel_apply(lambda x: get_dicom_meta(x, DESIRED_ATTRS), axis=1)

print(f"... Processing Completed in  >>>>> {time.time()-t1} <<<<<  Seconds ...\n")

display(dicom_df)

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">4.6  PERFORM ANALYSIS ON EACH COLUMN</h2>

---

We will first identify all of the possible values (including NaN) for a given attribute as well as their respective occurence counts.

In [ ]:
for attr in DESIRED_ATTRS:
    print(f"\n\nAttribute Name: {attr} ({dicom_df[attr].astype(str).nunique()} Different Values)\n")
    for i, (k,v) in enumerate(dicom_df[attr].astype(str).value_counts(dropna=False).items()): 
        print(f"\t{repr(k):<50} --> Occurs {v} Times In Training Data")
        if i == 10:
            print("\n\t--------- TRUNCATED ------ ENDING EARLY ------ TRUNCATED ---------")
            break

<br>

<a id="helper_functions"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="3d_numpy">5&nbsp;&nbsp;LOAD 3D REPRESENTATIONS OF EXAMPLES</h1>

I have gone ahead and created 3D numpy arrays containing the images interpolated to a fixed size (512,512,128) or (128,128,32). Please see this notebook for details on how this was completed.

<br>

**NOTE SOME IDS DO NOT POSSESS RESPECTIVE SCANS WITH NON-ZERO VALUES**
* FLAIR --> ['00709.npz', '00109.npz']
* T1w   --> ['00123.npz']
* T1wCE --> []
* T2w   --> ['00123.npz']

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">5.1  SETUP FILE PATHS</h2>

---

In [ ]:
# train_ids = [f"{x:>05}.npz" for x in os.listdir("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train")]

# print([x for x in train_ids if x not in os.listdir(FLAIR_NPY_DIR)])
# print([x for x in train_ids if x not in os.listdir(T1w_NPY_DIR)])
# print([x for x in train_ids if x not in os.listdir(T1wCE_NPY_DIR)])
# print([x for x in train_ids if x not in os.listdir(T2w_NPY_DIR)])

# total_sum = 0
# IDX = "00026"
# TYPE = "T2w"
# for i in range(1000):
#     try:
#         total_sum+=pydicom.read_file([os.path.join(f"/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{IDX}/{TYPE}", x) for x in os.listdir(f"/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{IDX}/{TYPE}")][i]).pixel_array.max()
#     except:
#         break
        
# print(total_sum)

In [ ]:
TRAIN_NPY_DIR = "/kaggle/input/create-3d-npz-rsna-radiogenomic-classification/train"
# TRAIN_NPY_DIR = "/kaggle/input/rsna-radiogenomic-3dnumpy-512x512x128/train"
TEST_NPY_DIR = "/kaggle/input/create-3d-npz-rsna-radiogenomic-classification/test"

TRAIN_FLAIR_NPY_DIR = os.path.join(TRAIN_NPY_DIR, "FLAIR")
TRAIN_T1w_NPY_DIR = os.path.join(TRAIN_NPY_DIR, "T1w")
TRAIN_T1wCE_NPY_DIR = os.path.join(TRAIN_NPY_DIR, "T1wCE")
TRAIN_T2w_NPY_DIR = os.path.join(TRAIN_NPY_DIR, "T2w")

TEST_FLAIR_NPY_DIR = os.path.join(TEST_NPY_DIR, "FLAIR")
TEST_T1w_NPY_DIR = os.path.join(TEST_NPY_DIR, "T1w")
TEST_T1wCE_NPY_DIR = os.path.join(TEST_NPY_DIR, "T1wCE")
TEST_T2w_NPY_DIR = os.path.join(TEST_NPY_DIR, "T2w")


print("\n... FILE COUNTS ...\n")
print(f"\t--> The number of FLAIR numpy files is:  {len(os.listdir(TRAIN_FLAIR_NPY_DIR))}  ...")
print(f"\t--> The number of T1w   numpy files is:  {len(os.listdir(TRAIN_T1w_NPY_DIR))}  ...")
print(f"\t--> The number of T1wCE numpy files is:  {len(os.listdir(TRAIN_T1wCE_NPY_DIR))}  ...")
print(f"\t--> The number of T2w   numpy files is:  {len(os.listdir(TRAIN_T2w_NPY_DIR))}  ...\n")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">5.2  UPDATE THE TRAIN DATAFRAME</h2>

---

In [ ]:
train_df["path_to_flair_npz"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_FLAIR_NPY_DIR, f"{x:>05}.npz"))
train_df["path_to_t1w_npz"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_T1w_NPY_DIR, f"{x:>05}.npz"))
train_df["path_to_t1wce_npz"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_T1wCE_NPY_DIR, f"{x:>05}.npz"))
train_df["path_to_t2w_npz"] = train_df.BraTS21ID.apply(lambda x: os.path.join(TRAIN_T2w_NPY_DIR, f"{x:>05}.npz"))

ss_df["path_to_flair_npz"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_FLAIR_NPY_DIR, f"{x:>05}.npz"))
ss_df["path_to_t1w_npz"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_T1w_NPY_DIR, f"{x:>05}.npz"))
ss_df["path_to_t1wce_npz"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_T1wCE_NPY_DIR, f"{x:>05}.npz"))
ss_df["path_to_t2w_npz"] = ss_df.BraTS21ID.apply(lambda x: os.path.join(TEST_T2w_NPY_DIR, f"{x:>05}.npz"))

display(train_df.head())
display(ss_df.head())

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">5.3  VISUALIZE A RANDOM SCAN</h2>

---

In [ ]:
RANDOM_IMG_IDX = int(random.random()*len(train_df))
print(f"\n... Visualize Single Flair Scan From Disk - IDX={RANDOM_IMG_IDX} ...\n")
display(create_animation(load_npz(train_df.path_to_flair_npz[RANDOM_IMG_IDX])))

RANDOM_IMG_IDX = int(random.random()*len(train_df))
print(f"\n... Visualize Set of mpMRI Scans From Disk - IDX={RANDOM_IMG_IDX} ...\n")
display(create_4scan_animation(
    load_npz(train_df.path_to_flair_npz[RANDOM_IMG_IDX]),
    load_npz(train_df.path_to_t1w_npz[RANDOM_IMG_IDX]),
    load_npz(train_df.path_to_t1wce_npz[RANDOM_IMG_IDX]),
    load_npz(train_df.path_to_t2w_npz[RANDOM_IMG_IDX]),
))

print(train_df.path_to_flair_npz[RANDOM_IMG_IDX])
print(train_df.path_to_t1w_npz[RANDOM_IMG_IDX])
print(train_df.path_to_t1wce_npz[RANDOM_IMG_IDX])
print(train_df.path_to_t2w_npz[RANDOM_IMG_IDX])

In [ ]:
# def get_3d_effnet(dropout=0.5):
#     """Build a 3D convolutional neural network model."""

#     inputs = tf.keras.layers.Input(shape=INPUT_SHAPE)
#     pp_inputs = tf.keras.applications.efficientnet.preprocess_input(inputs)
#     x = efn.EfficientNetB0(input_shape=INPUT_SHAPE, pooling="avg")(pp_inputs)
    
#     x = tf.keras.layers.Dropout(dropout)(x)
#     x = tf.keras.layers.Dense(units=256, activation="relu")(x)

#     x = tf.keras.layers.Dropout(dropout/1.25)(x)
#     outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

#     # Define the model.
#     model = tf.keras.Model(inputs, outputs, name="3d_mpmri_cnn")
#     return model

# model = get_3d_effnet()
# print(model.predict(tf.repeat(load_npz(train_df.path_to_flair_npz[RANDOM_IMG_IDX]).reshape(1, *INPUT_SHAPE[:-1], 1), axis=-1, repeats=3)))


<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #CD0001; background-color: #ffffff;" id="modelling">6&nbsp;&nbsp;MODELLING</h1>

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.1 CREATE THE MODELS</h2>

---

<br>

One for each type of scan

<br>

In [ ]:
INPUT_SHAPE = (128,128,32,1)
BATCH_SIZE = 8

def get_3d_model(width=128, height=128, depth=32, dropout=0.3):
    """Build a 3D convolutional neural network model."""

    inputs = tf.keras.layers.Input(shape=(width, height, depth, 1))

    # print(x.shape)
    # x1 = tf.keras.layers.Conv3D(filters=64, kernel_size=1, activation=None, padding="same")(inputs)
    # x1 = tf.keras.layers.MaxPool3D(pool_size=2)(x1)
    # x1 = tf.keras.layers.BatchNormalization()(x1)
    
    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # x3 = tf.keras.layers.Conv3D(filters=64, kernel_size=5, activation=None, padding="same")(inputs)
    # x3 = tf.keras.layers.MaxPool3D(pool_size=2)(x3)
    # x3 = tf.keras.layers.BatchNormalization()(x3)
    # x = tf.keras.layers.Add()([x1,x2,x3])
    # x = tf.keras.layers.Activation("relu")(x)
    
    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # print(x.shape)
    x = tf.keras.layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    # x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # print(x.shape)
    x = tf.keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    # x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout/2)(x)
    
    # print(x.shape)
    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(units=128, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)

    # print(x.shape)
    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = tf.keras.Model(inputs, outputs, name="3d_mpmri_cnn")
    return model


# Build model.
DROPOUT = 0.666
flair_model = get_3d_model(width=INPUT_SHAPE[0], height=INPUT_SHAPE[1], depth=INPUT_SHAPE[2], dropout=DROPOUT)
t1w_model = get_3d_model(width=INPUT_SHAPE[0], height=INPUT_SHAPE[1], depth=INPUT_SHAPE[2], dropout=DROPOUT)
t1wce_model = get_3d_model(width=INPUT_SHAPE[0], height=INPUT_SHAPE[1], depth=INPUT_SHAPE[2], dropout=DROPOUT)
t2w_model = get_3d_model(width=INPUT_SHAPE[0], height=INPUT_SHAPE[1], depth=INPUT_SHAPE[2], dropout=DROPOUT)
models = {"flair":flair_model, "t1w":t1w_model, "t1wce":t1wce_model, "t2w":t2w_model}

print("\n... EXAMPLE PREDICTION ...\n")
print(flair_model.predict(load_npz(train_df.path_to_flair_npz[RANDOM_IMG_IDX]).reshape(1, *INPUT_SHAPE)))

print("\n\n... MODEL SUMMARY ...\n")
flair_model.summary()

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.2  CREATE THE DATASETS</h2>

---

In [ ]:
# This is dataframe wrangling
USE_COL = "all" # one of [`all`, `path_to_t1w_npz`, `path_to_t1wce_npz`, `path_to_flair_npz`, `path_to_t2w_npz`]
if USE_COL is "all":
    USE_COLS = ["path_to_t1w_npz", "path_to_t1wce_npz", "path_to_flair_npz", "path_to_t2w_npz"]
else:
    USE_COLS = [USE_COL,]

# Initialize dictionaries
dataset_storage = {}

for use_col in USE_COLS:
    col_identifier = use_col.split("_")[2]
    dataset_storage[col_identifier] = {}
    
    dataset_storage[col_identifier]["train_ds_df"] = train_df[train_df[use_col].apply(lambda x: True if os.path.isfile(x) else False)][["MGMT_value", use_col]]
    dataset_storage[col_identifier]["train_lbl_list"] = dataset_storage[col_identifier]["train_ds_df"].MGMT_value.to_list()
    dataset_storage[col_identifier]["train_npz_file_list"] = dataset_storage[col_identifier]["train_ds_df"][use_col].to_list()

    # fix this.......
    dataset_storage[col_identifier]["test_ds_df"] = ss_df[ss_df[use_col].apply(lambda x: True if os.path.isfile(x) else False)][["BraTS21ID", use_col]]
    dataset_storage[col_identifier]["test_id_list"] = dataset_storage[col_identifier]["test_ds_df"].BraTS21ID.to_list()
    dataset_storage[col_identifier]["test_npz_file_list"] = dataset_storage[col_identifier]["test_ds_df"][use_col].to_list()

    # This is for splitting
    dataset_storage[col_identifier]["N_EX"] = len(dataset_storage[col_identifier]["train_lbl_list"])
    dataset_storage[col_identifier]["VAL_FRAC"] = 0.1
    dataset_storage[col_identifier]["N_TRAIN"] = int(dataset_storage[col_identifier]["N_EX"]*(1-dataset_storage[col_identifier]["VAL_FRAC"]))
    dataset_storage[col_identifier]["N_VAL"] = dataset_storage[col_identifier]["N_EX"]-dataset_storage[col_identifier]["N_TRAIN"]
    dataset_storage[col_identifier]["RANDOM_INDICES"] = random.sample(range(dataset_storage[col_identifier]["N_EX"]), dataset_storage[col_identifier]["N_EX"])
    dataset_storage[col_identifier]["TRAIN_INDICES"] = dataset_storage[col_identifier]["RANDOM_INDICES"][:dataset_storage[col_identifier]["N_TRAIN"]]
    dataset_storage[col_identifier]["VAL_INDICES"] = dataset_storage[col_identifier]["RANDOM_INDICES"][dataset_storage[col_identifier]["N_TRAIN"]:]

    # TF Data Integration
    dataset_storage[col_identifier]["lbl_train_ds"] = tf.data.Dataset.from_tensor_slices(np.array(dataset_storage[col_identifier]["train_lbl_list"])[dataset_storage[col_identifier]["TRAIN_INDICES"]])
    dataset_storage[col_identifier]["npz_file_train_ds"] = tf.data.Dataset.from_tensor_slices(np.array(dataset_storage[col_identifier]["train_npz_file_list"])[dataset_storage[col_identifier]["TRAIN_INDICES"]])
    dataset_storage[col_identifier]["raw_train_ds"] = tf.data.Dataset.zip((dataset_storage[col_identifier]["npz_file_train_ds"], dataset_storage[col_identifier]["lbl_train_ds"]))

    dataset_storage[col_identifier]["lbl_val_ds"] = tf.data.Dataset.from_tensor_slices(np.array(dataset_storage[col_identifier]["train_lbl_list"])[dataset_storage[col_identifier]["VAL_INDICES"]])
    dataset_storage[col_identifier]["npz_file_val_ds"] = tf.data.Dataset.from_tensor_slices(np.array(dataset_storage[col_identifier]["train_npz_file_list"])[dataset_storage[col_identifier]["VAL_INDICES"]])
    dataset_storage[col_identifier]["raw_val_ds"] = tf.data.Dataset.zip((dataset_storage[col_identifier]["npz_file_val_ds"], dataset_storage[col_identifier]["lbl_val_ds"]))

    dataset_storage[col_identifier]["id_test_ds"] = tf.data.Dataset.from_tensor_slices(dataset_storage[col_identifier]["test_id_list"])
    dataset_storage[col_identifier]["npz_file_test_ds"] = tf.data.Dataset.from_tensor_slices(dataset_storage[col_identifier]["test_npz_file_list"])
    dataset_storage[col_identifier]["raw_test_ds"] = tf.data.Dataset.zip((dataset_storage[col_identifier]["npz_file_test_ds"], dataset_storage[col_identifier]["id_test_ds"]))

    if INPUT_SHAPE[-1]==3:
        dataset_storage[col_identifier]["train_ds"] = dataset_storage[col_identifier]["raw_train_ds"].map(lambda x,y: (tf.repeat(tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), (*INPUT_SHAPE[:-1], 1))/255, axis=-1, repeats=3), tf.cast(y, tf.uint8)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["train_ds"] = dataset_storage[col_identifier]["train_ds"].shuffle(BATCH_SIZE*5).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        dataset_storage[col_identifier]["val_ds"] = dataset_storage[col_identifier]["raw_val_ds"].map(lambda x,y: (tf.repeat(tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), (*INPUT_SHAPE[:-1], 1))/255, axis=-1, repeats=3), tf.cast(y, tf.uint8)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["val_ds"] = dataset_storage[col_identifier]["val_ds"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        dataset_storage[col_identifier]["test_ds"] = dataset_storage[col_identifier]["raw_test_ds"].map(lambda x,y: (tf.repeat(tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), (*INPUT_SHAPE[:-1], 1))/255, axis=-1, repeats=3), tf.cast(y, tf.int32)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["test_ds"] = dataset_storage[col_identifier]["test_ds"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    else:
        dataset_storage[col_identifier]["train_ds"] = dataset_storage[col_identifier]["raw_train_ds"].map(lambda x,y: (tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), INPUT_SHAPE)/255, tf.cast(y, tf.uint8)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["train_ds"] = dataset_storage[col_identifier]["train_ds"].shuffle(BATCH_SIZE*5).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        dataset_storage[col_identifier]["val_ds"] = dataset_storage[col_identifier]["raw_val_ds"].map(lambda x,y: (tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), INPUT_SHAPE)/255, tf.cast(y, tf.uint8)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["val_ds"] = dataset_storage[col_identifier]["val_ds"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        dataset_storage[col_identifier]["test_ds"] = dataset_storage[col_identifier]["raw_test_ds"].map(lambda x,y: (tf.reshape(tf.py_function(
            load_npz, [x, True], (tf.uint8,)
        ), INPUT_SHAPE)/255, tf.cast(y, tf.int32)), num_parallel_calls=tf.data.AUTOTUNE)
        dataset_storage[col_identifier]["test_ds"] = dataset_storage[col_identifier]["test_ds"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for k, v in dataset_storage.items():
    print(f"\n\n... SCAN TYPE: {k.upper()}")
    print("\t-->", v["train_ds"])
    print("\t-->", v["val_ds"])
    print("\t-->", v["test_ds"])
print("\n")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.3  VISUALIZE A FEW EXAMPLES</h2>

---

In [ ]:
def plot_batch(dataset):
    r_idx = int(random.random()*BATCH_SIZE)
    
    img, lbl = next(iter(dataset))
    
    img = tf.squeeze(img[r_idx])
    lbl = lbl[r_idx]
    
    plt.figure(figsize=(20,10.5), )
    plt.suptitle(f"LABEL = {label_map_int_2_str[lbl.numpy()]} ({lbl.numpy()})", fontsize=20, fontweight="bold")
    for i in range(img.shape[-1]):
        plt.subplot(4,8,i+1)
        plt.imshow(img[..., i], cmap="gray")
        plt.axis(False)
    
    plt.tight_layout(h_pad=0.025, w_pad=0.025)
    # plt.subplots_adjust(wspace=0.001, hspace=0.001)
    plt.show()
    

for k, v in dataset_storage.items():
    print("\n\n\n\n\n\n"+"-"*35)
    print("-"*35)
    print(f"\t SCAN TYPE: {k.upper()}")
    print("-"*35)
    print("-"*35,"\n\n")
    
    print(f"\n\n... DEMO TRAIN {k.upper()} EXAMPLE ...\n\n")
    plot_batch(v["train_ds"])
    
    print(f"\n\n... DEMO VALIDATION {k.upper()} EXAMPLE ...\n\n")
    plot_batch(v["val_ds"])

print("\n")

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.4  LEARNING RATE FUNCTION/PLOT</h2>

---

In [ ]:
# Learning Rate
MAX_STEPS = 20000
INIT_LR = 0.00015
DECAY_RATE = 0.98
DECAY_STEPS = int((len(train_df)/BATCH_SIZE)*5) # 5 epochs = 1 decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    INIT_LR, decay_steps=DECAY_STEPS, decay_rate=DECAY_RATE, staircase=True
)

# Plot Learning Rate
plt.figure(figsize=(10,10))
plt.plot(range(0, MAX_STEPS, int(len(train_df)/BATCH_SIZE)), [lr_schedule(i) for i in range(0, MAX_STEPS, int(len(train_df)/BATCH_SIZE))], linewidth=3)
plt.vlines(range(int(len(train_df)/BATCH_SIZE*5), MAX_STEPS, int(len(train_df)/BATCH_SIZE*5)), ymin=0, ymax=INIT_LR, linestyles="dotted", colors='gray')
plt.vlines(range(int(len(train_df)/BATCH_SIZE*50), MAX_STEPS, int(len(train_df)/BATCH_SIZE*50)), ymin=0, ymax=INIT_LR, colors='r')
plt.title("Learning Rate\nDotted Lines At Every 5 Epochs (~350 steps)\nSolid Red Lines At Every 50 Epochs", fontweight="bold")
plt.show()

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.5  TRAINING LOOP</h2>

---

In [ ]:
# Compile model
N_EPOCHS=250
PATIENCE=50

histories = {}
for model_style, model in models.items():
    print("\n\n\n\n\n\n\n\n")
    print("-"*80)
    print("-"*20+"\t"+f"{model_style.upper()} TRAINING LOOP"+"\t"+"-"*20)
    print("-"*80)
    print("\n\n\n")
    
    # Get datasets
    train_ds = dataset_storage[model_style]["train_ds"]
    val_ds = dataset_storage[model_style]["val_ds"]
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        INIT_LR, decay_steps=DECAY_STEPS, decay_rate=DECAY_RATE, staircase=True
    )

    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc", tf.keras.metrics.AUC(name="auc")],
    )

    # Define callbacks.
    CKPT_DIR = os.path.join("/kaggle/working/model_files", model_style)
    if not os.path.isdir(CKPT_DIR): 
        os.makedirs(CKPT_DIR, exist_ok=True)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        os.path.join(CKPT_DIR, "epoch_{epoch:02d}--val_loss_{val_loss:.2f}--val_auc_{val_auc:.2f}"), 
        save_best_only=True, monitor="val_auc", mode="max",
    )
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_auc", patience=PATIENCE, restore_best_weights=True, mode="max")
    
    # Train the model, doing validation at the end of each epoch
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=N_EPOCHS,
        callbacks=[checkpoint_cb, early_stopping_cb],
    )
    histories[model_style] = history

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.6  PLOT MODEL TRAINING</h2>

---

hTBD

In [ ]:
histories

<br>

<h2 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #CD0001; background-color: #ffffff;">6.7  MODEL PREDICTIONS</h2>

---

In [ ]:
model_preds_dfs = {}
for model_style, model in tqdm(models.items(), total=4):
    model_preds_dfs[model_style] = pd.DataFrame()
    model_preds_dfs[model_style]["BraTS21ID"] = flatten_l_o_l([x.tolist() for x in list(dataset_storage[model_style]["test_ds"].map(lambda x,y: y).as_numpy_iterator())])
    model_preds_dfs[model_style][model_style+"_MGMT_value"] = tf.squeeze(model.predict(dataset_storage[model_style]["test_ds"].map(lambda x,y: x)))
    
ss_df = ss_df[["BraTS21ID"]]
for style, style_df in model_preds_dfs.items():
    ss_df = pd.merge(ss_df, style_df, on="BraTS21ID", how="left")
ss_df[["BraTS21ID", "flair_MGMT_value", "t1w_MGMT_value", "t1wce_MGMT_value", "t2w_MGMT_value"]].to_csv("submission.csv", index=False)
display(ss_df)